# LLM基礎実習: Google Colab版

このノートブックは、Google ColabでLLMの基本的な使い方を学ぶためのものです。

## 学習目標
- Google ColabでLLM環境をセットアップする
- 簡単なLLMモデルを読み込んで使用する
- プロンプトエンジニアリングの基礎を理解する
- テキスト生成の基本的な仕組みを体験する

## 注意事項
- このノートブックはGoogle Colabで実行することを前提としています
- GPUを使用する場合は、ランタイム > ランタイムのタイプを変更 > GPU を選択してください
- 無料版では制限がありますが、基本的な学習には十分です


## 0. 環境セットアップ

まず、必要なライブラリをインストールして環境をセットアップします。


In [ ]:
# 必要なライブラリのインストール
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets accelerate evaluate scikit-learn pandas numpy matplotlib seaborn tqdm

# インストール確認
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("GPU name:", torch.cuda.get_device_name(0))


In [ ]:
# 必要なライブラリのインポート
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')

# 日本語フォントの設定（可視化用）
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.style.use('default')

print("✅ ライブラリのインポートが完了しました")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


## 1. 簡単なLLMモデルの読み込み

まずは、比較的軽量なGPT-2モデルを使ってみましょう。Colabでは、モデルのダウンロードに時間がかかる場合があります。


In [ ]:
# モデルとトークナイザーの読み込み
model_name = "gpt2"  # 軽量なGPT-2モデル

print(f"🔄 {model_name} を読み込み中...")
print("初回実行時は、モデルのダウンロードに時間がかかります。")

# トークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained(model_name)

# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(model_name)

# パディングトークンを設定
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ モデルの読み込みが完了しました！")
print(f"📊 モデルパラメータ数: {model.num_parameters():,}")
print(f"🔤 語彙数: {len(tokenizer)}")

# デバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"🖥️ 使用デバイス: {device}")


## 2. 基本的なテキスト生成

シンプルなプロンプトからテキストを生成してみましょう。


In [ ]:
def generate_text(prompt, max_length=100, temperature=0.7, top_p=0.9):
    """
    テキストを生成する関数
    
    Args:
        prompt (str): 入力プロンプト
        max_length (int): 最大生成長
        temperature (float): 生成のランダム性（0.0-1.0）
        top_p (float): 核サンプリングのパラメータ
    """
    # テキストをトークン化
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    # テキスト生成
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    # 生成されたテキストをデコード
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# テスト用のプロンプト
test_prompts = [
    "The future of artificial intelligence is",
    "Once upon a time, there was a",
    "In the world of machine learning,",
    "The secret to success is",
    "Climate change is one of the most"
]

print("=== 基本的なテキスト生成 ===")
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- プロンプト {i} ---")
    print(f"入力: {prompt}")
    generated = generate_text(prompt, max_length=80)
    print(f"出力: {generated}")
    print("-" * 50)


## 3. パラメータの調整実験

異なるパラメータで生成結果がどう変わるか実験してみましょう。


In [ ]:
# 温度パラメータの比較実験
prompt = "The secret to success is"

print("=== 温度パラメータの比較 ===")
temperatures = [0.3, 0.7, 1.0]

results = []
for temp in temperatures:
    print(f"\n--- 温度: {temp} ---")
    temp_results = []
    for i in range(3):  # 同じプロンプトで3回生成
        generated = generate_text(prompt, max_length=60, temperature=temp)
        print(f"{i+1}: {generated}")
        temp_results.append(generated)
    results.append((temp, temp_results))

# 結果の可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Temperature Parameter Comparison', fontsize=16)

for i, (temp, temp_results) in enumerate(results):
    axes[i].text(0.1, 0.5, f'Temp: {temp}\n\n' + '\n\n'.join(temp_results), 
                transform=axes[i].transAxes, fontsize=10, 
                verticalalignment='center', wrap=True)
    axes[i].set_title(f'Temperature = {temp}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()


## 4. プロンプトエンジニアリングの基礎

プロンプトの書き方で生成結果が大きく変わります。様々なテクニックを試してみましょう。


In [ ]:
# プロンプトエンジニアリングの例
print("=== プロンプトエンジニアリングの例 ===\n")

# 異なるプロンプトスタイル
prompts = {
    "シンプル": "Write a story about a robot.",
    "具体的": "Write a creative science fiction story about a robot who discovers emotions.",
    "例示付き": "Write a story about a robot. Here's an example: Once upon a time, there was a robot who...",
    "役割指定": "You are a creative writer. Write an engaging story about a robot.",
    "構造化": """Write a story about a robot with the following structure:
1. Introduction: Introduce the robot
2. Conflict: What challenge does it face?
3. Resolution: How does it solve the problem?
4. Conclusion: What does it learn?"""
}

results = {}
for style, prompt in prompts.items():
    print(f"--- {style} ---")
    print(f"プロンプト: {prompt}")
    generated = generate_text(prompt, max_length=120, temperature=0.8)
    print(f"生成結果: {generated}")
    results[style] = generated
    print("-" * 80)


## 5. Few-shot学習の実践

Few-shot学習では、例を示すことでモデルの性能を向上させることができます。


In [ ]:
# Few-shot学習の例：感情分析
def few_shot_sentiment_analysis(text, examples):
    """Few-shot感情分析"""
    prompt = "Classify the sentiment of the following texts as positive, negative, or neutral:\n\n"
    
    # 例を追加
    for example_text, example_label in examples:
        prompt += f"Text: {example_text}\nSentiment: {example_label}\n\n"
    
    # 対象テキストを追加
    prompt += f"Text: {text}\nSentiment:"
    
    return prompt

# 感情分析の例
sentiment_examples = [
    ("I love this product! It's amazing!", "positive"),
    ("This is terrible. I hate it.", "negative"),
    ("The weather is okay today.", "neutral"),
    ("Fantastic! Best experience ever!", "positive"),
    ("I'm so disappointed with this service.", "negative")
]

# テスト用のテキスト
test_texts = [
    "This movie was absolutely incredible!",
    "I'm not sure how I feel about this.",
    "Worst purchase I've ever made.",
    "It's fine, I guess.",
    "Outstanding quality and great value!"
]

print("=== Few-shot感情分析 ===")
for text in test_texts:
    print(f"\nテキスト: {text}")
    
    # Few-shotプロンプトの生成
    few_shot_prompt = few_shot_sentiment_analysis(text, sentiment_examples)
    
    # 生成実行
    result = generate_text(few_shot_prompt, max_length=50, temperature=0.3)
    print(f"Few-shot結果: {result}")
    print("-" * 60)


## 6. インタラクティブなテキスト生成

ユーザーが自由にプロンプトを入力できるインタラクティブな機能を作成しましょう。


In [ ]:
# インタラクティブなテキスト生成
def interactive_generation():
    """インタラクティブなテキスト生成"""
    print("🤖 インタラクティブなLLMテキスト生成")
    print("プロンプトを入力してください（終了するには 'quit' と入力）")
    print("=" * 50)
    
    while True:
        try:
            # プロンプトの入力
            prompt = input("\n💬 プロンプト: ")
            
            if prompt.lower() == 'quit':
                print("👋 終了します。お疲れ様でした！")
                break
            
            if not prompt.strip():
                print("⚠️ プロンプトを入力してください。")
                continue
            
            # パラメータの設定
            print("\n⚙️ パラメータ設定:")
            try:
                max_length = int(input("最大長 (デフォルト: 100): ") or "100")
                temperature = float(input("温度 (デフォルト: 0.7): ") or "0.7")
            except ValueError:
                print("⚠️ 無効な値です。デフォルト値を使用します。")
                max_length = 100
                temperature = 0.7
            
            # テキスト生成
            print(f"\n🔄 生成中... (長さ: {max_length}, 温度: {temperature})")
            generated = generate_text(prompt, max_length=max_length, temperature=temperature)
            
            print(f"\n✨ 生成結果:")
            print(f"{generated}")
            print("=" * 50)
            
        except KeyboardInterrupt:
            print("\n\n👋 終了します。お疲れ様でした！")
            break
        except Exception as e:
            print(f"❌ エラーが発生しました: {e}")
            continue

# インタラクティブ生成の開始
print("インタラクティブなテキスト生成を開始します...")
print("注意: Colabでは入力が制限される場合があります。")
print("ローカル環境で実行することをお勧めします。\n")

# サンプル実行（Colab用）
sample_prompts = [
    "Write a haiku about artificial intelligence:",
    "Explain quantum computing in simple terms:",
    "Create a recipe for chocolate cake:"
]

print("=== サンプル実行 ===")
for prompt in sample_prompts:
    print(f"\nプロンプト: {prompt}")
    generated = generate_text(prompt, max_length=80, temperature=0.7)
    print(f"生成結果: {generated}")
    print("-" * 60)


## 7. パフォーマンス分析と可視化

生成されたテキストの品質やパラメータの影響を分析してみましょう。


In [ ]:
# パフォーマンス分析
def analyze_generation_quality(prompt, num_samples=5, temperatures=[0.3, 0.7, 1.0]):
    """生成品質の分析"""
    results = {}
    
    for temp in temperatures:
        temp_results = []
        for i in range(num_samples):
            generated = generate_text(prompt, max_length=100, temperature=temp)
            temp_results.append(generated)
        results[temp] = temp_results
    
    return results

# 分析実行
prompt = "The future of technology will be"
print(f"分析プロンプト: {prompt}")
print("=" * 60)

quality_results = analyze_generation_quality(prompt, num_samples=3)

# 結果の表示と可視化
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Generation Quality Analysis by Temperature', fontsize=16)

for i, (temp, texts) in enumerate(quality_results.items()):
    # テキストの表示
    print(f"\n--- 温度 {temp} ---")
    for j, text in enumerate(texts, 1):
        print(f"{j}: {text}")
    
    # 可視化
    axes[i].text(0.05, 0.95, f'Temperature: {temp}', transform=axes[i].transAxes, 
                fontsize=14, fontweight='bold', verticalalignment='top')
    
    # テキストを表示（長い場合は省略）
    display_text = '\n\n'.join([text[:100] + '...' if len(text) > 100 else text for text in texts])
    axes[i].text(0.05, 0.85, display_text, transform=axes[i].transAxes, 
                fontsize=10, verticalalignment='top', wrap=True)
    axes[i].set_xlim(0, 1)
    axes[i].set_ylim(0, 1)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

# 統計情報
print("\n=== 統計情報 ===")
for temp, texts in quality_results.items():
    avg_length = np.mean([len(text.split()) for text in texts])
    print(f"温度 {temp}: 平均単語数 {avg_length:.1f}")


## 8. より大きなモデルの体験

GPT-2の他に、より大きなモデルも試してみましょう。Colabの制限内で利用可能なモデルを紹介します。


In [ ]:
# より大きなモデルの比較
def load_and_compare_models():
    """複数のモデルを比較"""
    models_to_try = [
        ("gpt2", "GPT-2 (117M parameters)"),
        ("gpt2-medium", "GPT-2 Medium (345M parameters)"),
        # ("gpt2-large", "GPT-2 Large (762M parameters)"),  # メモリ制限のためコメントアウト
        # ("gpt2-xl", "GPT-2 XL (1.5B parameters)")  # メモリ制限のためコメントアウト
    ]
    
    results = {}
    
    for model_name, description in models_to_try:
        try:
            print(f"\n🔄 {description} を読み込み中...")
            
            # トークナイザーとモデルの読み込み
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForCausalLM.from_pretrained(model_name)
            
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token
            
            # デバイスに移動
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            
            # テスト生成
            test_prompt = "The future of artificial intelligence is"
            generated = generate_text_with_model(model, tokenizer, test_prompt, device)
            
            results[model_name] = {
                'description': description,
                'parameters': model.num_parameters(),
                'generated': generated
            }
            
            print(f"✅ {description} 完了")
            print(f"パラメータ数: {model.num_parameters():,}")
            print(f"生成結果: {generated[:100]}...")
            
            # メモリ解放
            del model, tokenizer
            torch.cuda.empty_cache() if torch.cuda.is_available() else None
            
        except Exception as e:
            print(f"❌ {description} の読み込みに失敗: {e}")
            continue
    
    return results

def generate_text_with_model(model, tokenizer, prompt, device, max_length=80, temperature=0.7):
    """指定されたモデルでテキスト生成"""
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# モデル比較の実行
print("=== モデル比較実験 ===")
print("注意: 大きなモデルはメモリを多く使用します。")
print("Colabの制限により、一部のモデルは読み込めない場合があります。\n")

model_results = load_and_compare_models()

# 結果の可視化
if model_results:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    model_names = list(model_results.keys())
    param_counts = [model_results[name]['parameters'] for name in model_names]
    descriptions = [model_results[name]['description'] for name in model_names]
    
    bars = ax.bar(range(len(model_names)), param_counts, color=['skyblue', 'lightcoral', 'lightgreen', 'gold'][:len(model_names)])
    ax.set_xlabel('Model')
    ax.set_ylabel('Number of Parameters')
    ax.set_title('Model Size Comparison')
    ax.set_xticks(range(len(model_names)))
    ax.set_xticklabels([name.split('-')[0] for name in model_names], rotation=45)
    
    # パラメータ数をバーの上に表示
    for i, (bar, count) in enumerate(zip(bars, param_counts)):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(param_counts)*0.01,
                f'{count:,}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    # 生成結果の比較
    print("\n=== 生成結果の比較 ===")
    test_prompt = "The future of artificial intelligence is"
    for model_name, result in model_results.items():
        print(f"\n{result['description']}:")
        print(f"生成結果: {result['generated']}")
else:
    print("❌ 利用可能なモデルがありませんでした。")


## 9. 学習のまとめと次のステップ

このノートブックで学んだことをまとめて、次の学習ステップを確認しましょう。


In [ ]:
# 学習のまとめ
print("🎓 LLM基礎学習のまとめ")
print("=" * 50)

learned_concepts = [
    "✅ LLMの基本的な仕組みと使い方",
    "✅ プロンプトエンジニアリングの基礎",
    "✅ パラメータ調整（温度、top_p等）の影響",
    "✅ Few-shot学習の実践",
    "✅ インタラクティブなテキスト生成",
    "✅ パフォーマンス分析と可視化",
    "✅ 複数モデルの比較実験"
]

print("このノートブックで学習した内容:")
for concept in learned_concepts:
    print(f"  {concept}")

print("\n🔍 重要なポイント:")
key_points = [
    "• プロンプトの書き方が生成品質に大きく影響する",
    "• 温度パラメータで創造性を調整できる",
    "• Few-shot学習で特定のタスクに特化できる",
    "• より大きなモデルほど高品質な生成が可能",
    "• パラメータの調整は用途に応じて行う"
]

for point in key_points:
    print(f"  {point}")

print("\n📚 次の学習ステップ:")
next_steps = [
    "1. より高度なプロンプトエンジニアリング",
    "2. ファインチューニングの実践",
    "3. 感情分析や分類タスクの実装",
    "4. より大きなモデル（GPT-3.5/4、Claude等）の使用",
    "5. マルチモーダル（画像+テキスト）の学習",
    "6. LLMコンペティションへの参加"
]

for step in next_steps:
    print(f"  {step}")

print("\n💡 実践的なアドバイス:")
advice = [
    "• 毎日少しずつでもLLMを使い続ける",
    "• 新しいプロンプトテクニックを試してみる",
    "• コミュニティ（Discord、Twitter等）に参加する",
    "• 実際のプロジェクトでLLMを活用する",
    "• 最新の研究論文を読んで知識を更新する"
]

for tip in advice:
    print(f"  {tip}")

print("\n🌟 おめでとうございます！")
print("LLMの基礎学習が完了しました。")
print("これからは実践を通じてさらにスキルを向上させていきましょう！")


## 10. 追加リソースと参考資料

### 有用なリソース
- **Hugging Face Hub**: https://huggingface.co/models
- **Transformers ライブラリ**: https://huggingface.co/docs/transformers/
- **OpenAI API**: https://platform.openai.com/
- **Anthropic Claude**: https://www.anthropic.com/

### 学習に役立つ書籍・論文
- "Attention Is All You Need" (Transformer論文)
- "Language Models are Few-Shot Learners" (GPT-3論文)
- "Training language models to follow instructions" (InstructGPT論文)

### コミュニティ
- **Hugging Face Discord**: 最新の技術情報交換
- **r/MachineLearning**: Redditの機械学習コミュニティ
- **Twitter**: #LLM #NLP ハッシュタグで最新情報をフォロー

### 実践的なプロジェクトアイデア
1. **チャットボット開発**: 特定のドメインに特化した対話システム
2. **テキスト要約ツール**: 長い文章を自動で要約
3. **翻訳アプリ**: 多言語対応の翻訳システム
4. **コンテンツ生成**: ブログ記事やマーケティング文書の自動生成
5. **コード生成支援**: プログラミングの補助ツール

---

**このノートブックを完了したら、ぜひGitHubに保存して、他の人と共有してください！**
